<h2> Quotes to Scrape</h2>
here we will scrape <a> http://quotes.toscrape.com</a> for author and quotes information. We will store this data in SQLiteDatabase and instead using records libray we will be using dataset libray.<br>
<i>pip install dataset</i><br><br>
<b>Dataset library</b><br>
Although managing data in relational databases has plenty of benefits, they’re rarely used in day-to-day work with small to medium scale datasets. But why is that? Why do we see an awful lot of data stored in static files in CSV or JSON format, even though they are hard to query and update incrementally? And in Python, a database isn’t the simplest solution for storing a bunch of structured data. This is what dataset is going to change! <b>dataset</b> provides a simple abstraction layer that removes most direct SQL statements without the necessity for a full ORM model - essentially, databases can be used like a JSON file or NoSQL store.

In [1]:
import requests
import dataset
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
db = dataset.connect('sqlite:///C://Users//Qamar Abbas//Desktop//Learning//pws4ds//sqlite-databases//quotes_scrape.db')
authors_seen = set()

def scrape_quotes(html_soup):
    for quote in html_soup.select('div.quote'):
        quote_text = quote.find(class_='text').get_text(strip=True)
        quote_author_url = clean_url(quote.find(class_='author').find_next_sibling('a').get('href'))
        #list of tags
        quote_tag_urls = [
            clean_url(a.get('href'))for a in quote.find_all('a', class_='tag')
            ]
        authors_seen.add(quote_author_url)
        
        #dataset creates a table quotes and inserts data as dictionary. Here key represents colmun name 
        #creates id column and assigns it a PRIMARY KEY automaticaly 
        quote_id = db['quotes'].insert({ 
            'text' : quote_text,
            'author' : quote_author_url })
        #dataset creates a table quote_tags and inserts a list of tags
        db['quote_tags'].insert_many([{
            'quote_id' : quote_id, 
            'tag_id' : tag} for tag in quote_tag_urls])

def clean_url(url):
    # Clean '/author/Steve-Martin' to 'Steve-Martin'
    # Use urljoin to make an absolute URL
    url = urljoin(base_url, url) # (http://quotes.toscrape.com/,/author/Albert-Einstein/) -> http://quotes.toscrape.com/author/Albert-Einstein/
    # Use urlparse to get out the path part
    path = urlparse(url).path #returns path after base url 
    # Now split the path by '/' 
    # split return list ['', 'author', 'Steve-Martin'] and get the second part
    return path.split('/')[2]

def scrape_author(html_soup, author_id):
    author_name = html_soup.find(class_='author-title').get_text(strip=True)
    author_born_date = html_soup.find(class_='author-born-date').get_text(strip=True)
    author_born_loc = html_soup.find(class_='author-born-location').get_text(strip=True)
    author_desc = html_soup.find(class_='author-description').get_text(strip=True)
    db['authors'].insert({ 
'author_id' : author_id,
'name' : author_name,
'born_date' : author_born_date,
'born_location' : author_born_loc,
'description' : author_desc})

base_url = 'http://quotes.toscrape.com/'
# Start by scraping all the quote pages
url = base_url
while True:
    print('Now scraping page:', url)
    r = requests.get(url)
    html_soup = BeautifulSoup(r.text, 'html.parser')
    # Scrape the quotes
    scrape_quotes(html_soup)

    # Is there a next page?
    next_a = html_soup.select('li.next > a')
    if not next_a or not next_a[0].get('href'):break
    url = urljoin(url, next_a[0].get('href'))

# Now fetch out the author information
for author_id in authors_seen:
    url = urljoin(base_url, '/author/' + author_id)
    print('Now scraping author:', url)
    r = requests.get(url)
    html_soup = BeautifulSoup(r.text, 'html.parser')
    # Scrape the author information
    scrape_author(html_soup, author_id)

Now scraping page: http://quotes.toscrape.com/
Now scraping page: http://quotes.toscrape.com/page/2/
Now scraping page: http://quotes.toscrape.com/page/3/
Now scraping page: http://quotes.toscrape.com/page/4/
Now scraping page: http://quotes.toscrape.com/page/5/
Now scraping page: http://quotes.toscrape.com/page/6/
Now scraping page: http://quotes.toscrape.com/page/7/
Now scraping page: http://quotes.toscrape.com/page/8/
Now scraping page: http://quotes.toscrape.com/page/9/
Now scraping page: http://quotes.toscrape.com/page/10/
Now scraping author: http://quotes.toscrape.com/author/Harper-Lee
Now scraping author: http://quotes.toscrape.com/author/Helen-Keller
Now scraping author: http://quotes.toscrape.com/author/Jimi-Hendrix
Now scraping author: http://quotes.toscrape.com/author/Ralph-Waldo-Emerson
Now scraping author: http://quotes.toscrape.com/author/Bob-Marley
Now scraping author: http://quotes.toscrape.com/author/Thomas-A-Edison
Now scraping author: http://quotes.toscrape.com/auth

In [5]:
str='/author/Steve-Martin'
print(str.split('/'))

['', 'author', 'Steve-Martin']
